In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from geopy.geocoders import GoogleV3
from geopy.exc import GeocoderTimedOut
import folium


In [ ]:
# A function to find newspaper articles with 'kidnap' search query
def find_articles(start, end):
    data = []
    keyword = 'flood'

    # home Url of newspaper to scrape from
    base_url = 'https://punchng.com/page/'

    # Keyword integration
    key= '/?s='+ keyword

    for pageNo in range(start, end):
        # combining url with page number
        url = base_url+ str(pageNo)+key
        html_text= requests.get(url).text
        soap = BeautifulSoup(html_text,'html.parser')
        articles =soap.find_all('article')
        # print(articles)

        for article in articles:
            title = article.find_all('h1',class_='post-title')
            if title is not None:
                for i in title:
                    title = i.text.strip()
                    links= i.a['href']
                    article_response = requests.get(links)
                    article_soup = BeautifulSoup(article_response.text, 'html.parser')
                    news_=article_soup.find_all('div',class_='post-content')
                    pub_date =article_soup.find('span',class_='post-date').text.strip()
                    for news_article in news_:
                        news = news_article.find('p')
                        if news is not None:
                        # print(newss.text.strip())
                            data.append({'pub_date':pub_date,'headline': title, 'article': news.text.strip()})
    return data
df=pd.DataFrame(find_articles(1,100))
# print(df['Title'])
# print(df.head())
df.to_excel('naija_flood.xlsx')
# print('fin')

In [ ]:
df.shape

(979, 3)

In [ ]:
df

,pub_date,headline,article
0,4th May 2023,"Rwanda floods, landslides kill 130",Rwandans grieved Thursday for lost loved ones ...
1,4th May 2023,Obalende mega pumping station will curb floodi...,Lagos State Government on Wednesday said the m...
2,3rd May 2023,127 die as flood hits Rwanda,At least 127 people have died as floods and la...
3,3rd May 2023,"13,000 Nigerians benefited from flood relief i...","The Federal Government says over 13,000 Nigeri..."
4,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...
...,...,...,...
974,25th April 2019,Death toll in South Africa flood hits 70,Around 70 people have been killed in devastati...
975,24th April 2019,South African president pledges to mobilise re...,President Cyril Ramaphosa of South Africa on W...
976,24th April 2019,"51 killed, 1,000 people displaced by floods in...",Devastating floods in South Africa have left 5...
977,24th April 2019,51 die in South Africa floods,Heavy rains have lashed South Africa in recent...


# Data Cleaning

In [ ]:
def clean(text):
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))

    return text

In [ ]:
df['clean'] = df['article'].apply(clean)

In [ ]:
df

,pub_date,headline,article,clean
0,4th May 2023,"Rwanda floods, landslides kill 130",Rwandans grieved Thursday for lost loved ones ...,Rwandans grieved Thursday for lost loved ones ...
1,4th May 2023,Obalende mega pumping station will curb floodi...,Lagos State Government on Wednesday said the m...,Lagos State Government on Wednesday said the m...
2,3rd May 2023,127 die as flood hits Rwanda,At least 127 people have died as floods and la...,At least 127 people have died as floods and la...
3,3rd May 2023,"13,000 Nigerians benefited from flood relief i...","The Federal Government says over 13,000 Nigeri...","The Federal Government says over 13,000 Nigeri..."
4,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,Following a downpour in parts of Lagos and Ogu...
...,...,...,...,...
974,25th April 2019,Death toll in South Africa flood hits 70,Around 70 people have been killed in devastati...,Around 70 people have been killed in devastati...
975,24th April 2019,South African president pledges to mobilise re...,President Cyril Ramaphosa of South Africa on W...,President Cyril Ramaphosa of South Africa on W...
976,24th April 2019,"51 killed, 1,000 people displaced by floods in...",Devastating floods in South Africa have left 5...,Devastating floods in South Africa have left 5...
977,24th April 2019,51 die in South Africa floods,Heavy rains have lashed South Africa in recent...,Heavy rains have lashed South Africa in recent...


# Split the articles into Sentences

In [ ]:
# split sentences
def sentences(text):
    # split sentences and questions
    text = re.split('[.?]', text)
    clean_sent = []
    for sent in text:
        clean_sent.append(sent)
    return clean_sent

In [ ]:
# sentences
df['sentence'] = df['clean'].apply(sentences)

In [ ]:
df.head()

,pub_date,headline,article,clean,sentence
0,4th May 2023,"Rwanda floods, landslides kill 130",Rwandans grieved Thursday for lost loved ones ...,Rwandans grieved Thursday for lost loved ones ...,[Rwandans grieved Thursday for lost loved ones...
1,4th May 2023,Obalende mega pumping station will curb floodi...,Lagos State Government on Wednesday said the m...,Lagos State Government on Wednesday said the m...,[Lagos State Government on Wednesday said the ...
2,3rd May 2023,127 die as flood hits Rwanda,At least 127 people have died as floods and la...,At least 127 people have died as floods and la...,[At least 127 people have died as floods and l...
3,3rd May 2023,"13,000 Nigerians benefited from flood relief i...","The Federal Government says over 13,000 Nigeri...","The Federal Government says over 13,000 Nigeri...","[The Federal Government says over 13,000 Niger..."
4,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,Following a downpour in parts of Lagos and Ogu...,[Following a downpour in parts of Lagos and Og...


In [ ]:
# create a dataframe containing sentences
df2 = pd.DataFrame(columns=['sentence','date','len','headline'])

row_list = []

for i in range(len(df)):
    for sent in df.loc[i,'sentence']:
        headline = df.loc[i, 'headline']
        year = df.loc[i,'pub_date']

        dict1 = {'date':year,'headline':headline,'sentence':sent}
        row_list.append(dict1)

df2 = pd.DataFrame(row_list)

In [ ]:
df2

,date,headline,sentence
0,4th May 2023,"Rwanda floods, landslides kill 130",Rwandans grieved Thursday for lost loved ones ...
1,4th May 2023,"Rwanda floods, landslides kill 130",The government was still counting the cost a...
2,4th May 2023,"Rwanda floods, landslides kill 130",Rivers of mud swept away homes and other infr...
3,4th May 2023,"Rwanda floods, landslides kill 130",“I found my child buried under rocks and bric...
4,4th May 2023,"Rwanda floods, landslides kill 130","He died at the hospital,” Anonciata, who only..."
...,...,...,...
21678,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",“We expect the death toll to increase as some...
21679,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",“It’s really bad
21680,23rd April 2019,"17 killed in Uganda after heavy rains, flooding","” In March 2010, at least 100 people died in s..."
21681,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",Efforts by Uganda’s government over the years...


## Classifying Sentences into Flooded or Non-flooded based on the Headline

In [ ]:
def classify_text(text):
    # List of flooded keywords
    flooded_keywords = ['displace', 'kill', 'destroy','submerge','stranded',
                        'sweep','fix','collapse','washes','take over','lost',
                        'defeat', 'ruined', 'overtake', 'die','death',
                        'missing','submerged','ravage','hits','losing','sacks','diversion','cuts off',
                        'overuns','saves','worsen']

    # Check if any of the flooded keywords are present in the text
    for keyword in flooded_keywords:
        if keyword in text.lower():
            return 'flooded'

    # If none of the flooded keywords are present, return non-flooded
    return 'non-flooded'

In [ ]:
df2['status'] = df2['headline'].apply(classify_text)

In [ ]:
df2

,date,headline,sentence,status
0,4th May 2023,"Rwanda floods, landslides kill 130",Rwandans grieved Thursday for lost loved ones ...,flooded
1,4th May 2023,"Rwanda floods, landslides kill 130",The government was still counting the cost a...,flooded
2,4th May 2023,"Rwanda floods, landslides kill 130",Rivers of mud swept away homes and other infr...,flooded
3,4th May 2023,"Rwanda floods, landslides kill 130",“I found my child buried under rocks and bric...,flooded
4,4th May 2023,"Rwanda floods, landslides kill 130","He died at the hospital,” Anonciata, who only...",flooded
...,...,...,...,...
21678,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",“We expect the death toll to increase as some...,flooded
21679,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",“It’s really bad,flooded
21680,23rd April 2019,"17 killed in Uganda after heavy rains, flooding","” In March 2010, at least 100 people died in s...",flooded
21681,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",Efforts by Uganda’s government over the years...,flooded


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21683 entries, 0 to 21682
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      21683 non-null  object
 1   headline  21683 non-null  object
 2   sentence  21683 non-null  object
 3   status    21683 non-null  object
dtypes: object(4)
memory usage: 677.7+ KB


In [ ]:
# Filter the dataframe to contain only the flooded rows
flood = df2[df2['status'] == 'flooded'].reset_index(drop=True)

In [ ]:
flood.head()

,date,headline,sentence,status
0,4th May 2023,"Rwanda floods, landslides kill 130",Rwandans grieved Thursday for lost loved ones ...,flooded
1,4th May 2023,"Rwanda floods, landslides kill 130",The government was still counting the cost a...,flooded
2,4th May 2023,"Rwanda floods, landslides kill 130",Rivers of mud swept away homes and other infr...,flooded
3,4th May 2023,"Rwanda floods, landslides kill 130",“I found my child buried under rocks and bric...,flooded
4,4th May 2023,"Rwanda floods, landslides kill 130","He died at the hospital,” Anonciata, who only...",flooded


In [ ]:
flood.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7141 entries, 0 to 7140
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      7141 non-null   object
 1   headline  7141 non-null   object
 2   sentence  7015 non-null   object
 3   status    7141 non-null   object
dtypes: object(4)
memory usage: 223.3+ KB


In [ ]:
flood.dropna(subset ='sentence',inplace = True)

In [ ]:
flood.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7015 entries, 0 to 7140
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      7015 non-null   object
 1   headline  7015 non-null   object
 2   sentence  7015 non-null   object
 3   status    7015 non-null   object
dtypes: object(4)
memory usage: 274.0+ KB


In [ ]:
flood.reset_index(drop=True)

,date,headline,sentence,status
0,4th May 2023,"Rwanda floods, landslides kill 130",Rwandans grieved Thursday for lost loved ones ...,flooded
1,4th May 2023,"Rwanda floods, landslides kill 130",The government was still counting the cost a...,flooded
2,4th May 2023,"Rwanda floods, landslides kill 130",Rivers of mud swept away homes and other infr...,flooded
3,4th May 2023,"Rwanda floods, landslides kill 130",“I found my child buried under rocks and bric...,flooded
4,4th May 2023,"Rwanda floods, landslides kill 130","He died at the hospital,” Anonciata, who only...",flooded
...,...,...,...,...
7010,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",“We expect the death toll to increase as some...,flooded
7011,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",“It’s really bad,flooded
7012,23rd April 2019,"17 killed in Uganda after heavy rains, flooding","” In March 2010, at least 100 people died in s...",flooded
7013,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",Efforts by Uganda’s government over the years...,flooded


# Geoparsing using Dependency Parsing

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_locations(text):
    doc = nlp(text)
    locations = []
    for ent in doc.ents:
        if ent.label_ in ["GPE", "LOC","FAC","ORG","PERSON"]:
            if "," in ent.text:
                location = ent.text.split(",")[0]
            else:
                location = ent.text
            locations.append(location)
        else:
            for token in doc:
                if token.pos_ == "PROPN" and token.dep_ == "nsubj":
                    locations.append(token.text)
    return list(set(list(locations)))

flood['locations'] = flood['sentence'].apply(extract_locations)


In [ ]:
extract_locations(flood.iloc[292,2])

['Onibata', 'Nobel Street', 'Ogba', 'Lagos', 'PUNCH Metro', 'Metro', 'Okoro']

In [ ]:
for ent in nlp(flood.iloc[248,2]).ents:
  print(ent.text, ent.label_)

Aniagwu PERSON
Six CARDINAL
Oko Community GPE


In [ ]:
flood

,date,headline,sentence,status,locations
0,4th May 2023,"Rwanda floods, landslides kill 130",Rwandans grieved Thursday for lost loved ones ...,flooded,[]
1,4th May 2023,"Rwanda floods, landslides kill 130",The government was still counting the cost a...,flooded,[]
2,4th May 2023,"Rwanda floods, landslides kill 130",Rivers of mud swept away homes and other infr...,flooded,"[Western Province, Lake Kivu]"
3,4th May 2023,"Rwanda floods, landslides kill 130",“I found my child buried under rocks and bric...,flooded,[]
4,4th May 2023,"Rwanda floods, landslides kill 130","He died at the hospital,” Anonciata, who only...",flooded,"[Anonciata, AFP, Karongi]"
...,...,...,...,...,...
7136,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",“We expect the death toll to increase as some...,flooded,[]
7137,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",“It’s really bad,flooded,[]
7138,23rd April 2019,"17 killed in Uganda after heavy rains, flooding","” In March 2010, at least 100 people died in s...",flooded,[Bududa]
7139,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",Efforts by Uganda’s government over the years...,flooded,[Uganda]


In [ ]:
data= flood.explode("locations", ignore_index = True)

In [ ]:
data.shape

(12353, 5)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12353 entries, 0 to 12352
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       12353 non-null  object
 1   headline   12353 non-null  object
 2   sentence   12353 non-null  object
 3   status     12353 non-null  object
 4   locations  9000 non-null   object
dtypes: object(5)
memory usage: 482.7+ KB


In [ ]:
data.dropna(subset ='locations',inplace = True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9000 entries, 2 to 12351
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       9000 non-null   object
 1   headline   9000 non-null   object
 2   sentence   9000 non-null   object
 3   status     9000 non-null   object
 4   locations  9000 non-null   object
dtypes: object(5)
memory usage: 421.9+ KB


In [ ]:
data

,date,headline,sentence,status,locations
2,4th May 2023,"Rwanda floods, landslides kill 130",Rivers of mud swept away homes and other infr...,flooded,Western Province
3,4th May 2023,"Rwanda floods, landslides kill 130",Rivers of mud swept away homes and other infr...,flooded,Lake Kivu
5,4th May 2023,"Rwanda floods, landslides kill 130","He died at the hospital,” Anonciata, who only...",flooded,Anonciata
6,4th May 2023,"Rwanda floods, landslides kill 130","He died at the hospital,” Anonciata, who only...",flooded,AFP
7,4th May 2023,"Rwanda floods, landslides kill 130","He died at the hospital,” Anonciata, who only...",flooded,Karongi
...,...,...,...,...,...
12344,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",READ ALSO: UAE launches ‘Ministry of Possibil...,flooded,Uganda
12345,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",READ ALSO: UAE launches ‘Ministry of Possibil...,flooded,UAE
12346,23rd April 2019,"17 killed in Uganda after heavy rains, flooding",READ ALSO: UAE launches ‘Ministry of Possibil...,flooded,Bududa
12350,23rd April 2019,"17 killed in Uganda after heavy rains, flooding","” In March 2010, at least 100 people died in s...",flooded,Bududa


In [ ]:
data.to_excel("dataa.xlsx")

# GEOCODING

In [ ]:
data = pd.read_excel("dataa.xlsx")

In [ ]:
data

,date,headline,sentence,status,locations
0,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,flooded,the Lagos Ibadan Expressway
1,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,flooded,Ogun
2,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,flooded,the Long Bridge
3,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Related News NEMA alerts Kwara on 2023 flo...,flooded,Lagos State
4,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Related News NEMA alerts Kwara on 2023 flo...,flooded,Lagos Ibadan Expressway
...,...,...,...,...,...
7098,23rd June 2019,Flood sweeps away two teenagers in Delta,Testimony was a JSS 1 pupil of Agbor Technica...,flooded,Agbor Technical College
7099,23rd June 2019,Flood sweeps away two teenagers in Delta,SUNDAY PUNCH gathered that the bereaved fathe...,flooded,Revd
7100,23rd June 2019,Flood sweeps away two teenagers in Delta,"Chosen Godday, received a distress call by a ...",flooded,Godday
7101,23rd June 2019,Flood sweeps away two teenagers in Delta,"At Uromi Junction axis, I found a crowd who r...",flooded,Uromi Junction


In [ ]:
# Mount the Google Drive to load files from Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Specify the file path to the API key
file_path = '/content/drive/My Drive/google_API.txt'

In [ ]:
# Open the file and read the API key
with open(file_path, 'r') as f:
    api_key = next(f).strip()

In [ ]:

# Create a geocoder object with the API key
geolocator = GoogleV3(api_key=api_key)

In [ ]:

geolocator = GoogleV3(api_key=api_key, domain='maps.google.com.ng')

In [ ]:

# Function to geocode an address
def geocode_location(address):
    # address = ' '.join(addresses)
    try:
        if address:
          # Append "Nigeria" to the address
          full_address = f"{address}, Nigeria"
            # Use geopy to get the location's latitude and longitude
          geo_loc = geolocator.geocode(address)
          if not(geo_loc):
              return None
          return geo_loc # (geo_loc.latitude, geo_loc.longitude, geo_loc)
    except GeocoderTimedOut:
        # If geocoding times out, return None
        return None

In [ ]:
data['geo_locs'] = data['locations'].apply(geocode_location)

In [ ]:
data

,date,headline,sentence,status,locations,geo_locs
0,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,flooded,the Lagos Ibadan Expressway,"(Lagos - Ibadan Expy, Nigeria, (6.9235877, 3.6..."
2,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,flooded,the Long Bridge,"(The Long Bridge, Serenity Lee Trail, Sagle, I..."
3,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Related News NEMA alerts Kwara on 2023 flo...,flooded,Lagos State,"(Lagos, Nigeria, (6.522704399999999, 3.6217802))"
4,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Related News NEMA alerts Kwara on 2023 flo...,flooded,Lagos Ibadan Expressway,"(Lagos - Ibadan Expy, Nigeria, (6.9235877, 3.6..."
5,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,"Meanwhile, the Ogun State Commissioner for En...",flooded,Ogun States,"(Ogun State, Nigeria, (6.997974699999999, 3.47..."
...,...,...,...,...,...,...
7092,23rd June 2019,Flood sweeps away two teenagers in Delta,OCHEI MATTHEW Alibioba Community at Agbor Tech...,flooded,the Ika South Local Government Area,"(Ika South, Delta, Nigeria, (6.2650871, 6.1739..."
7094,23rd June 2019,Flood sweeps away two teenagers in Delta,OCHEI MATTHEW Alibioba Community at Agbor Tech...,flooded,Agbor Technical College,"(75FP+JX6, 321103, Agbor, Delta, Nigeria, (6.2..."
7096,23rd June 2019,Flood sweeps away two teenagers in Delta,Testimony was a JSS 1 pupil of Agbor Technica...,flooded,Alibioba Primary School,"(Paranas, Samar, Philippines, (11.9324579, 125..."
7098,23rd June 2019,Flood sweeps away two teenagers in Delta,Testimony was a JSS 1 pupil of Agbor Technica...,flooded,Agbor Technical College,"(75FP+JX6, 321103, Agbor, Delta, Nigeria, (6.2..."


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3270 entries, 0 to 7101
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       3270 non-null   object
 1   headline   3270 non-null   object
 2   sentence   3270 non-null   object
 3   status     3270 non-null   object
 4   locations  3270 non-null   object
 5   geo_locs   3268 non-null   object
dtypes: object(6)
memory usage: 178.8+ KB


In [ ]:
data.dropna(subset = 'geo_locs',inplace = True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3268 entries, 0 to 7101
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       3268 non-null   object
 1   headline   3268 non-null   object
 2   sentence   3268 non-null   object
 3   status     3268 non-null   object
 4   locations  3268 non-null   object
 5   geo_locs   3268 non-null   object
dtypes: object(6)
memory usage: 178.7+ KB


In [ ]:
data

,date,headline,sentence,status,locations,geo_locs
0,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,flooded,the Lagos Ibadan Expressway,"(Lagos - Ibadan Expy, Nigeria, (6.9235877, 3.6..."
2,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,flooded,the Long Bridge,"(The Long Bridge, Serenity Lee Trail, Sagle, I..."
3,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Related News NEMA alerts Kwara on 2023 flo...,flooded,Lagos State,"(Lagos, Nigeria, (6.522704399999999, 3.6217802))"
4,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Related News NEMA alerts Kwara on 2023 flo...,flooded,Lagos Ibadan Expressway,"(Lagos - Ibadan Expy, Nigeria, (6.9235877, 3.6..."
5,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,"Meanwhile, the Ogun State Commissioner for En...",flooded,Ogun States,"(Ogun State, Nigeria, (6.997974699999999, 3.47..."
...,...,...,...,...,...,...
7092,23rd June 2019,Flood sweeps away two teenagers in Delta,OCHEI MATTHEW Alibioba Community at Agbor Tech...,flooded,the Ika South Local Government Area,"(Ika South, Delta, Nigeria, (6.2650871, 6.1739..."
7094,23rd June 2019,Flood sweeps away two teenagers in Delta,OCHEI MATTHEW Alibioba Community at Agbor Tech...,flooded,Agbor Technical College,"(75FP+JX6, 321103, Agbor, Delta, Nigeria, (6.2..."
7096,23rd June 2019,Flood sweeps away two teenagers in Delta,Testimony was a JSS 1 pupil of Agbor Technica...,flooded,Alibioba Primary School,"(Paranas, Samar, Philippines, (11.9324579, 125..."
7098,23rd June 2019,Flood sweeps away two teenagers in Delta,Testimony was a JSS 1 pupil of Agbor Technica...,flooded,Agbor Technical College,"(75FP+JX6, 321103, Agbor, Delta, Nigeria, (6.2..."


In [ ]:
data['lat'] = data['geo_locs'].apply(lambda x: x.latitude)
data['lng'] = data['geo_locs'].apply(lambda x: x.longitude)
data['address'] = data['geo_locs'].apply(lambda x: x.address)

In [ ]:
data

,date,headline,sentence,status,locations,geo_locs,lat,lng,address
0,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,flooded,the Lagos Ibadan Expressway,"(Lagos - Ibadan Expy, Nigeria, (6.9235877, 3.6...",6.923588,3.636422,"Lagos - Ibadan Expy, Nigeria"
2,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,flooded,the Long Bridge,"(The Long Bridge, Serenity Lee Trail, Sagle, I...",48.252848,-116.538660,"The Long Bridge, Serenity Lee Trail, Sagle, ID..."
3,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Related News NEMA alerts Kwara on 2023 flo...,flooded,Lagos State,"(Lagos, Nigeria, (6.522704399999999, 3.6217802))",6.522704,3.621780,"Lagos, Nigeria"
4,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,Related News NEMA alerts Kwara on 2023 flo...,flooded,Lagos Ibadan Expressway,"(Lagos - Ibadan Expy, Nigeria, (6.9235877, 3.6...",6.923588,3.636422,"Lagos - Ibadan Expy, Nigeria"
5,2nd May 2023,Motorists stranded as flood takes over Lagos-I...,"Meanwhile, the Ogun State Commissioner for En...",flooded,Ogun States,"(Ogun State, Nigeria, (6.997974699999999, 3.47...",6.997975,3.473738,"Ogun State, Nigeria"
...,...,...,...,...,...,...,...,...,...
7092,23rd June 2019,Flood sweeps away two teenagers in Delta,OCHEI MATTHEW Alibioba Community at Agbor Tech...,flooded,the Ika South Local Government Area,"(Ika South, Delta, Nigeria, (6.2650871, 6.1739...",6.265087,6.173912,"Ika South, Delta, Nigeria"
7094,23rd June 2019,Flood sweeps away two teenagers in Delta,OCHEI MATTHEW Alibioba Community at Agbor Tech...,flooded,Agbor Technical College,"(75FP+JX6, 321103, Agbor, Delta, Nigeria, (6.2...",6.274040,6.187378,"75FP+JX6, 321103, Agbor, Delta, Nigeria"
7096,23rd June 2019,Flood sweeps away two teenagers in Delta,Testimony was a JSS 1 pupil of Agbor Technica...,flooded,Alibioba Primary School,"(Paranas, Samar, Philippines, (11.9324579, 125...",11.932458,125.038818,"Paranas, Samar, Philippines"
7098,23rd June 2019,Flood sweeps away two teenagers in Delta,Testimony was a JSS 1 pupil of Agbor Technica...,flooded,Agbor Technical College,"(75FP+JX6, 321103, Agbor, Delta, Nigeria, (6.2...",6.274040,6.187378,"75FP+JX6, 321103, Agbor, Delta, Nigeria"


In [ ]:
# Define the bounds of Nigeria
min_lon = 2.6769
max_lon = 14.6774
min_lat = 4.2672
max_lat = 13.8856

# Define a function to check if a row is within the bounds of Nigeria
def is_within_nigeria_bounds(row):
    return row['lng'] >= min_lon and row['lng'] <= max_lon and row['lat'] >= min_lat and row['lat'] <= max_lat

# Drop rows that are not within the bounds of Nigeria
data2 = data[data.apply(is_within_nigeria_bounds, axis=1)]


In [ ]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2251 entries, 0 to 7101
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       2251 non-null   object 
 1   headline   2251 non-null   object 
 2   sentence   2251 non-null   object 
 3   status     2251 non-null   object 
 4   locations  2251 non-null   object 
 5   geo_locs   2251 non-null   object 
 6   lat        2251 non-null   float64
 7   lng        2251 non-null   float64
 8   address    2251 non-null   object 
dtypes: float64(2), object(7)
memory usage: 175.9+ KB


In [ ]:
data2['year'] = data2['date'].dt.year

<ipython-input-35-a651bf501562>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['year'] = data2['date'].dt.year


In [ ]:
data2.reset_index()

,index,date,headline,sentence,status,locations,geo_locs,lat,lng,address,year
0,0,2023-05-02,Motorists stranded as flood takes over Lagos-I...,Following a downpour in parts of Lagos and Ogu...,flooded,the Lagos Ibadan Expressway,"(Lagos - Ibadan Expy, Nigeria, (6.9235877, 3.6...",6.923588,3.636422,"Lagos - Ibadan Expy, Nigeria",2023
1,3,2023-05-02,Motorists stranded as flood takes over Lagos-I...,Related News NEMA alerts Kwara on 2023 flo...,flooded,Lagos State,"(Lagos, Nigeria, (6.522704399999999, 3.6217802))",6.522704,3.621780,"Lagos, Nigeria",2023
2,4,2023-05-02,Motorists stranded as flood takes over Lagos-I...,Related News NEMA alerts Kwara on 2023 flo...,flooded,Lagos Ibadan Expressway,"(Lagos - Ibadan Expy, Nigeria, (6.9235877, 3.6...",6.923588,3.636422,"Lagos - Ibadan Expy, Nigeria",2023
3,5,2023-05-02,Motorists stranded as flood takes over Lagos-I...,"Meanwhile, the Ogun State Commissioner for En...",flooded,Ogun States,"(Ogun State, Nigeria, (6.997974699999999, 3.47...",6.997975,3.473738,"Ogun State, Nigeria",2023
4,7,2023-05-02,Motorists stranded as flood takes over Lagos-I...,"Meanwhile, the Ogun State Commissioner for En...",flooded,Ogun State,"(Ogun State, Nigeria, (6.997974699999999, 3.47...",6.997975,3.473738,"Ogun State, Nigeria",2023
...,...,...,...,...,...,...,...,...,...,...,...
2246,7085,2019-06-25,Obaseki deploys officials as flood hits Edo,"Osagie said, “The Special Adviser to the Gov...",flooded,Edo State Waste Management Board,"(Governor's Office, Oka 300102, Benin City, Ed...",6.320643,5.618453,"Governor's Office, Oka 300102, Benin City, Edo...",2019
2247,7092,2019-06-23,Flood sweeps away two teenagers in Delta,OCHEI MATTHEW Alibioba Community at Agbor Tech...,flooded,the Ika South Local Government Area,"(Ika South, Delta, Nigeria, (6.2650871, 6.1739...",6.265087,6.173912,"Ika South, Delta, Nigeria",2019
2248,7094,2019-06-23,Flood sweeps away two teenagers in Delta,OCHEI MATTHEW Alibioba Community at Agbor Tech...,flooded,Agbor Technical College,"(75FP+JX6, 321103, Agbor, Delta, Nigeria, (6.2...",6.274040,6.187378,"75FP+JX6, 321103, Agbor, Delta, Nigeria",2019
2249,7098,2019-06-23,Flood sweeps away two teenagers in Delta,Testimony was a JSS 1 pupil of Agbor Technica...,flooded,Agbor Technical College,"(75FP+JX6, 321103, Agbor, Delta, Nigeria, (6.2...",6.274040,6.187378,"75FP+JX6, 321103, Agbor, Delta, Nigeria",2019


## Visualizinf Locations on Map

In [ ]:
def plot_map(df, lat_col='lat', lon_col='lng', loc_name_col='address',headline="sentence"):
    # create map centered on the first location
    map_center = [df[lat_col].iloc[0], df[lon_col].iloc[0]]
    my_map = folium.Map(location=[9.0820, 8.6753], zoom_start=6)

    # add markers for each location in the dataframe
    for i, row in df.iterrows():
        location = [row[lat_col], row[lon_col]]

        loc_name = row[loc_name_col]
        headl = row[headline]
        popup = folium.Popup(headl, parse_html=True)
        folium.CircleMarker(location=location, radius=3, color='red', fill=True, fill_color='red', tooltip=loc_name,popup=popup).add_to(my_map)

    return my_map

In [ ]:
plot_map(data1, lat_col='lat', lon_col='lng', loc_name_col='address',headline="sentence")